## 1. install dependency

In [2]:
%pip install mindsdb mindsdb_sdk pandas requests datasets yaspin

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 2. Dataset Selection and Download



In [3]:
# Download IMDB Movies Dataset from Hugging Face
from datasets import load_dataset
import pandas as pd

# Load the dataset
print("Downloading IMDB Movies dataset...")
dataset = load_dataset("jquigl/imdb-genres")
df = pd.DataFrame(dataset["train"])

# Preview the dataset
print(f"Dataset shape: {df.shape}")
df.head()

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset shape: (238256, 5)


,movie title - year,genre,expanded-genres,rating,description
0,Flaming Ears - 1992,Fantasy,"Fantasy, Sci-Fi",6.0,Flaming Ears is a pop sci-fi lesbian fantasy f...
1,Jeg elsker dig - 1957,Romance,"Comedy, Drama, Romance",5.8,Six people - three couples - meet at random at...
2,Povjerenje - 2021,Thriller,Thriller,NaN,"In a small unnamed town, in year 2025, Krsto w..."
3,Gulliver Returns - 2021,Fantasy,"Animation, Adventure, Family",4.4,The legendary Gulliver returns to the Kingdom ...
4,Prithvi Vallabh - 1924,Biography,"Biography, Drama, Romance",NaN,"Seminal silent historical film, the story feat..."


Let's prepare our dataset for MindsDB by cleaning it up and making sure we have a unique ID column:

In [4]:
# Clean up the data and ensure we have a unique ID
# The 'movie title - year' column can serve as a unique identifier
df = df.rename(columns={
    'movie title - year': 'movie_id',
    'expanded-genres': 'expanded_genres',
    'description': 'content'
})

# Clean movie IDs to remove problematic characters
import re
def clean_movie_id(movie_id):
    if pd.isna(movie_id) or movie_id == '':
        return "unknown_movie"
    
    cleaned = str(movie_id)
    cleaned = re.sub(r"['\"\!\?\(\)\[\]\/\\*]", "", cleaned)
    cleaned = cleaned.replace("&", "and").replace(":", "_")
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
    return cleaned if cleaned else "unknown_movie"

# Apply the cleaning function to movie_id column
df['movie_id'] = df['movie_id'].apply(clean_movie_id)

# Remove duplicates based on cleaned movie_id, keeping the first occurrence
print(f"Original dataset size: {len(df)}")
df = df.drop_duplicates(subset=['movie_id'], keep='first')
print(f"After removing duplicates: {len(df)}")

# Make sure there are no NaN values
df = df.fillna({
    'movie_id': 'unknown_movie', 
    'genre': 'unknown', 
    'expanded_genres': '',
    'rating': 0.0, 
    'content': ''
})

# Save the prepared dataset
df.to_csv('imdb_movies_prepared.csv', index=False)
print("Dataset prepared and saved to 'imdb_movies_prepared.csv'")
df.head()

Original dataset size: 238256
After removing duplicates: 161765
Dataset prepared and saved to 'imdb_movies_prepared.csv'


,movie_id,genre,expanded_genres,rating,content
0,Flaming Ears - 1992,Fantasy,"Fantasy, Sci-Fi",6.0,Flaming Ears is a pop sci-fi lesbian fantasy f...
1,Jeg elsker dig - 1957,Romance,"Comedy, Drama, Romance",5.8,Six people - three couples - meet at random at...
2,Povjerenje - 2021,Thriller,Thriller,0.0,"In a small unnamed town, in year 2025, Krsto w..."
3,Gulliver Returns - 2021,Fantasy,"Animation, Adventure, Family",4.4,The legendary Gulliver returns to the Kingdom ...
4,Prithvi Vallabh - 1924,Biography,"Biography, Drama, Romance",0.0,"Seminal silent historical film, the story feat..."


## 3. Uploading the Dataset to MindsDB

Now let's connect to our local MindsDB instance and upload the dataset:

In [5]:
import mindsdb_sdk

# Connect to the MindsDB server
# For local Docker installation, use the default URL
server = mindsdb_sdk.connect('http://127.0.0.1:47334')
print("Connected to MindsDB server")

# List available databases to confirm connection
databases = server.databases.list()
print("Available databases:")
for db in databases:
    print(f"- {db.name}")

Connected to MindsDB server
Available databases:
- files
- pvec
- movies_kb_chromadb


Now let's create a file database to upload our prepared CSV:

In [6]:
import os, pandas as pd, mindsdb_sdk

# connect
server = mindsdb_sdk.connect("http://127.0.0.1:47334")

# load (or generate) the DataFrame
csv_path  = os.path.abspath("imdb_movies_prepared.csv")
csv_path2 = os.path.abspath("customer_support_tickets.csv")
df_movies = pd.read_csv(csv_path)

# upload to the built‑in  `files`  database
files_db   = server.get_database("files")            
table_name = "movies"

# delete the whole file‑table if it's there
try:
    files_db.tables.drop(table_name)
    print(f"dropped {table_name}")
except Exception:
    pass

files_db.create_table(table_name, df_movies)
print(f"Created table files.{table_name}")

print(
    server.query(
        f"SELECT movie_id, genre, rating FROM files.{table_name} LIMIT 5"
    ).fetch()
)

print(
    server.query(
        f"SELECT count(movie_id) FROM files.{table_name} where rating >= 7.5"
    ).fetch()
)


dropped movies
Created table files.movies
                  movie_id      genre  rating
0      Flaming Ears - 1992    Fantasy     6.0
1    Jeg elsker dig - 1957    Romance     5.8
2        Povjerenje - 2021   Thriller     0.0
3  Gulliver Returns - 2021    Fantasy     4.4
4   Prithvi Vallabh - 1924  Biography     0.0
   count_0
0    10152


## 4. Creating a Knowledge Base



In [13]:
# -- drop the KB if it exists --
server.query("DROP KNOWLEDGE_BASE IF EXISTS movies_kb;").fetch()

# Knowledge Base creation using mindsdb_sdk
try:
    kb_creation_query = server.query(f"""
    CREATE KNOWLEDGE_BASE movies_kb
    USING
        embedding_model = {{
           "provider": "ollama",
           "model_name": "nomic-embed-text"
        }},
        metadata_columns = ['genre', 'expanded_genres', 'rating'],
        content_columns = ['content'],
        id_column = 'movie_id';
    """)
    kb_creation_query.fetch()
    print("Created knowledge base 'movies_kb'")
except Exception as e:
    print(f"Knowledge base creation error or already exists: {e}")

Created knowledge base 'movies_kb'


Now let's insert our movie data into the knowledge base:

In [ ]:
from yaspin import yaspin
import traceback

try:
    df = server.query("SELECT COUNT(*) AS total FROM files.movies WHERE rating >= 7.5").fetch()
    total_rows = df["total"].iloc[0]  # ✅ safer access

    batch_size = 5000

    for offset in range(0, total_rows, batch_size):
        with yaspin(text=f"Inserting batch starting at offset {offset}..."):
            server.query(f"""
                INSERT INTO movies_kb
                SELECT movie_id,
                       genre,
                       expanded_genres,
                       rating,
                       content
                FROM files.movies
                WHERE rating >= 7.5
                LIMIT {batch_size} OFFSET {offset}
                USING track_column = movie_id
            """).fetch()
        print(f"✅ Batch inserted successfully: OFFSET {offset}")

    print("🎉 All data inserted successfully!")

except Exception as e:
    print(f"❌ Insert error: {type(e).__name__}: {e}")
    traceback.print_exc()


In [ ]:
import mindsdb_sdk
from yaspin import yaspin

server = mindsdb_sdk.connect('http://127.0.0.1:47334')

total_rows = 10152
batch_size = 5000

for offset in range(0, total_rows, batch_size):
    limit = min(batch_size, total_rows - offset)

    with yaspin(text=f"Inserting batch OFFSET {offset} LIMIT {limit}..."):
        try:
            query = f"""
            INSERT INTO movies_kb
            SELECT movie_id, genre, expanded_genres, rating, content
            FROM files.movies
            WHERE rating >= 7.5
            LIMIT {limit} OFFSET {offset}
            USING track_column = 'movie_id';
            """
            server.query(query)
        except Exception as e:
            print(f"❌ Error at offset {offset}: {e}")
            break


📦 Total rows to insert into KB: 10152


In [24]:
row_count_df = server.query("""
    SELECT COUNT(*) AS cnt
    FROM   (SELECT id FROM movies_kb) AS t;
""").fetch()

row_count = int(row_count_df.at[0, 'cnt'])
print(f"✅ movies_kb now contains {row_count:,} rows.")

✅ movies_kb now contains 10,152 rows.


Let's see some data in the knowledge base:

In [18]:
search_query = server.query("SELECT * FROM movies_kb where content='Christmas' order by relevance desc")
display(search_query.fetch())

,id,chunk_id,chunk_content,metadata,distance,relevance
0,Pixi Post and the Gift Bringers - 2016,Pixi Post and the Gift Bringers - 2016:content...,Christmas is on danger. Only Pixi Post can sav...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.269095,0.787963
1,A Christmas Carol - 1938,A Christmas Carol - 1938:content:1of1:0to63,An elderly miser learns the error of his ways ...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.295026,0.772185
2,Hope for the Holidays - 2020,Hope for the Holidays - 2020:content:1of1:0to192,One man's love for his dying mother leads him ...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.298542,0.770094
3,For Unto Us - 2021,For Unto Us - 2021:content:1of1:0to96,A musical portrayal of the nativity of Jesus C...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.308408,0.764287
4,Christmas Snow Angels - 2011,Christmas Snow Angels - 2011:content:1of1:0to160,A young girl is dealing with the death of a lo...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.313398,0.761384
5,Hodina modrych slonu - 1971,Hodina modrych slonu - 1971:content:1of1:0to222,"Two days before Christmas. Zuzanka and Tomás, ...","{""chunk_index"": 0, ""content_column"": ""content""...",0.322649,0.756059
6,Winter Thaw - 2016,Winter Thaw - 2016:content:1of1:0to103,An old man comes to realize the mistakes he ha...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.332027,0.750735
7,Project_ Puppies for Christmas - 2019,Project_ Puppies for Christmas - 2019:content:...,Two young girls attempt to bring Christmas joy...,"{""chunk_index"": 0, ""content_column"": ""content""...",0.343670,0.744230
8,A Very Quarantwinned Christmas - 2020,A Very Quarantwinned Christmas - 2020:content:...,"When a snowstorm strands their mom in Iowa, tw...","{""chunk_index"": 0, ""content_column"": ""content""...",0.349688,0.740912
9,Uninvited for the Holidays - 2022,Uninvited for the Holidays - 2022:content:1of1...,"A new age ""family"" Christmas comedy, a fish ou...","{""chunk_index"": 0, ""content_column"": ""content""...",0.352226,0.739521


## 5. Performing Semantic Searches

Now that our knowledge base is ready (or being populated), let's do some Q&A:

In [20]:
import pandas as pd

def search_kb(query: str, limit: int = 100) -> pd.DataFrame:
    try:
        result = server.query(f"""
            SELECT *
            FROM movies_kb
            WHERE MATCH('{query}')
            ORDER BY relevance DESC
            LIMIT {limit};
        """).fetch()
        return result
        
    except Exception as e:
        print(f"❌ KB search error: {e}")
        return pd.DataFrame()


In [24]:
import requests
import json
from IPython.display import display
import pandas as pd

OLLAMA_MODEL = "llama2"
OLLAMA_HOST = "http://localhost:11434"

# Dummy KB search function for now
def search_kb(query: str, limit: int = 100) -> pd.DataFrame:
    return pd.DataFrame({
        'id': [1],
        'chunk_content': [
            "In 'Home Alone', a boy defends his home against two burglars named Harry and Marv on Christmas Eve."
        ],
        'relevance': [0.99]
    })

# NEW: Streaming-safe response reader
def read_streamed_response(response):
    full_answer = ""
    for line in response.iter_lines():
        if line:
            try:
                obj = json.loads(line.decode('utf-8'))
                full_answer += obj.get("message", {}).get("content", "")
            except json.JSONDecodeError as e:
                print("⚠️ Skipping malformed line:", line)
    return full_answer.strip()

def answer_question_with_llm(question: str):
    print(f"Searching knowledge base for: '{question}'\n")

    # Step 1: Get relevant chunks
    relevant_chunks_df = search_kb(question, limit=100)
    print("Found the following relevant chunks:")
    display(relevant_chunks_df[['id', 'chunk_content', 'relevance']])

    # Step 2: Combine chunks into a single context
    context = "\n---\n".join(relevant_chunks_df['chunk_content'])

    # Step 3: Construct prompt
    prompt = f"""You are a movie expert assistant. Based *only* on the following movie summaries (context),
answer the user's question. If the context doesn't contain the answer,
state that you cannot answer based on the provided information.
Return only the plain answer as a single paragraph, with no JSON or markdown formatting.

CONTEXT:
{context}

QUESTION:
{question}
"""

    print(f"\nSending request to Ollama (model: {OLLAMA_MODEL}) to generate a definitive answer...")

    try:
        response = requests.post(
            f"{OLLAMA_HOST}/api/chat",
            json={
                "model": OLLAMA_MODEL,
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant that answers questions about movies using only the provided context."},
                    {"role": "user", "content": prompt}
                ],
                "stream": True,
                "temperature": 0.0
            },
            stream=True  # Required for streamed token responses
        )

        response.raise_for_status()

        print("📦 Reading streamed response...")
        answer = read_streamed_response(response)
        return answer

    except Exception as e:
        return f"❌ An error occurred while calling Ollama: {e}"

# Example usage
user_question = "Who a boy must defend his home against on Christmas eve?"
final_answer = answer_question_with_llm(user_question)

print("\n--- Generated Answer ---")
print(final_answer)


Searching knowledge base for: 'Who a boy must defend his home against on Christmas eve?'

Found the following relevant chunks:


,id,chunk_content,relevance
0,1,"In 'Home Alone', a boy defends his home agains...",0.99



Sending request to Ollama (model: llama2) to generate a definitive answer...
📦 Reading streamed response...

--- Generated Answer ---
In the movie "Home Alone," the boy must defend his home against the two burglars, Harry and Marv, on Christmas Eve.


In [ ]:
user_question = "What Anakin was lured into by Chancellor Palpatine?"
final_answer = answer_question_with_llm(user_question)

print("\n--- Generated Answer ---")
print(final_answer)

In [ ]:
model = server.models.get('llama2_movie_qa')

agent = server.agents.create(
    name='movie_expert_agent',
    model=model,
    knowledge_bases=['movies_kb'],
    prompt_template='''
        You are a movie expert assistant. Use the data in the knowledge base `movies_kb` (which contains plot summaries, genre, and metadata) to answer questions about movies.

        If the KB doesn’t contain enough context to answer the question, say “I cannot answer this based on the provided data.”
    '''
)
